In [13]:
#For reading and manipulating our data
import pandas as pd
import math
import random as rnd
from tabulate import tabulate

In [2]:
fights_df = pd.read_csv('drive/MyDrive/ufc_data/v2/fights_final')
fighters_df = pd.read_csv('drive/MyDrive/ufc_data/v2/ufc_db_fighters.csv')

In [3]:
for ind in fights_df.index:
  if "DECISION" in fights_df["method"][ind].upper():
    fights_df["method"][ind] = "DECISION"
  elif "SUBMISSION" in fights_df["method"][ind].upper():
    fights_df["method"][ind] = "SUBMISSION"
  elif "KO" in fights_df["method"][ind].upper():
    fights_df["method"][ind] = "KO"

<ipython-input-3-eb72f2970667>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fights_df["method"][ind] = "DECISION"
<ipython-input-3-eb72f2970667>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fights_df["method"][ind] = "SUBMISSION"
<ipython-input-3-eb72f2970667>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fights_df["method"][ind] = "KO"


In [4]:
def get_stats(fighter_name):
  fighter_fights_df = fights_df[(fights_df["fighter_1"] == fighter_name) | (fights_df["fighter_2"] == fighter_name)]

  if len(fighter_fights_df) == 0:
      return None

  fighter_stats = {"name": fighter_name, "total_fights": len(fighter_fights_df), "total_wins": 0, "total_losses": 0, "decision_wins":0,"ko_wins":0,"submission_wins":0,"decision_losses":0,"ko_losses":0,"submission_losses":0}

  for ind in fighter_fights_df.index:
    winner = fighter_fights_df['winner'][ind]+1

    if fighter_fights_df[f"fighter_{winner}"][ind]==fighter_name:
      result="wins" #FIGHTER WINS
    else:
      result="losses"

    fighter_stats[f"total_{result}"]+=1
    if fighter_fights_df["method"][ind] =="DECISION":
      fighter_stats[f"decision_{result}"]+=1
    elif fighter_fights_df["method"][ind] =="KO":
      fighter_stats[f"ko_{result}"]+=1
    elif fighter_fights_df["method"][ind] =="SUBMISSION":
      fighter_stats[f"submission_{result}"]+=1

  return fighter_stats

In [5]:
fighter_stats = []

for fighter in fighters_df["name"]:
  if stats := get_stats(fighter):
    fighter_stats.append(stats)

fighter_stats_df = pd.DataFrame(fighter_stats)

In [6]:
fighter_stats_df.set_index("name", inplace=True)
fighter_stats_df.head()

,total_fights,total_wins,total_losses,decision_wins,ko_wins,submission_wins,decision_losses,ko_losses,submission_losses
name,,,,,,,,,
Shamil Abdurakhimov,11,5,6,3,2,0,0,6,0
Daichi Abe,3,1,2,1,0,0,2,0,0
Klidson Abreu,3,1,2,1,0,0,2,0,0
Juan Adams,4,1,3,0,1,0,1,2,0
Zarrukh Adashev,4,1,3,1,0,0,1,2,0


In [7]:
def calculate_p(stat, total):
  return stat/total

def calculate_sd(stat, total):
  p = calculate_p(stat, total)
  return math.sqrt((p*(1-p))/total)

In [8]:
def calculate_fighter_params(fighter_name):
  params={
      "decision_wins_prop": calculate_p(fighter_stats_df.loc[fighter_name, "decision_wins"], fighter_stats_df.loc[fighter_name, "total_fights"]),
      "decision_wins_sd": calculate_sd(fighter_stats_df.loc[fighter_name, "decision_wins"], fighter_stats_df.loc[fighter_name, "total_fights"]),
      "ko_wins_prop": calculate_p(fighter_stats_df.loc[fighter_name, "ko_wins"], fighter_stats_df.loc[fighter_name, "total_fights"]),
      "ko_wins_sd": calculate_sd(fighter_stats_df.loc[fighter_name, "ko_wins"], fighter_stats_df.loc[fighter_name, "total_fights"]),
      "submission_wins_prop": calculate_p(fighter_stats_df.loc[fighter_name, "submission_wins"], fighter_stats_df.loc[fighter_name, "total_fights"]),
      "submission_wins_sd": calculate_sd(fighter_stats_df.loc[fighter_name, "submission_wins"], fighter_stats_df.loc[fighter_name, "total_fights"]),
      "decision_losses_prop": calculate_p(fighter_stats_df.loc[fighter_name, "decision_losses"], fighter_stats_df.loc[fighter_name, "total_fights"]),
      "decision_losses_sd": calculate_sd(fighter_stats_df.loc[fighter_name, "decision_losses"], fighter_stats_df.loc[fighter_name, "total_fights"]),
      "ko_losses_prop": calculate_p(fighter_stats_df.loc[fighter_name, "ko_losses"], fighter_stats_df.loc[fighter_name, "total_fights"]),
      "ko_losses_sd": calculate_sd(fighter_stats_df.loc[fighter_name, "ko_losses"], fighter_stats_df.loc[fighter_name, "total_fights"]),
      "submission_losses_prop": calculate_p(fighter_stats_df.loc[fighter_name, "submission_losses"], fighter_stats_df.loc[fighter_name, "total_fights"]),
      "submission_losses_sd": calculate_sd(fighter_stats_df.loc[fighter_name, "submission_losses"], fighter_stats_df.loc[fighter_name, "total_fights"])}

  return params


In [9]:
def get_bout_params(fighter_1, fighter_2):
  return pd.DataFrame([calculate_fighter_params(fighter_1), calculate_fighter_params(fighter_2)])

In [10]:
def single_fight_simulation(matchup_df):
  scores={}

  scores["fighter1_dec_score"] = (rnd.gauss(matchup_df.iloc[0]['decision_wins_prop'],matchup_df.iloc[0]['decision_wins_sd'])+
                            rnd.gauss(matchup_df.iloc[1]['decision_losses_prop'],matchup_df.iloc[1]['decision_losses_sd']))/2
  scores["fighter1_ko_score"] = (rnd.gauss(matchup_df.iloc[0]['ko_wins_prop'],matchup_df.iloc[0]['ko_wins_sd'])+
                            rnd.gauss(matchup_df.iloc[1]['ko_losses_prop'],matchup_df.iloc[1]['ko_losses_sd']))/2
  scores["fighter1_sub_score"] = (rnd.gauss(matchup_df.iloc[0]['submission_wins_prop'],matchup_df.iloc[0]['submission_wins_sd'])+
                            rnd.gauss(matchup_df.iloc[1]['submission_losses_prop'],matchup_df.iloc[1]['submission_losses_sd']))/2
  scores["fighter2_dec_score"] = (rnd.gauss(matchup_df.iloc[1]['decision_wins_prop'],matchup_df.iloc[1]['decision_wins_sd'])+
                            rnd.gauss(matchup_df.iloc[0]['decision_losses_prop'],matchup_df.iloc[0]['decision_losses_sd']))/2
  scores["fighter2_ko_score"] = (rnd.gauss(matchup_df.iloc[1]['ko_wins_prop'],matchup_df.iloc[1]['ko_wins_sd'])+
                            rnd.gauss(matchup_df.iloc[0]['ko_losses_prop'],matchup_df.iloc[0]['ko_losses_sd']))/2
  scores["fighter2_sub_score"] = (rnd.gauss(matchup_df.iloc[1]['submission_wins_prop'],matchup_df.iloc[1]['submission_wins_sd'])+
                            rnd.gauss(matchup_df.iloc[0]['submission_losses_prop'],matchup_df.iloc[0]['submission_losses_sd']))/2

  result=max(zip(scores.values(), scores.keys()))[1][:-6]

  return result


In [11]:
def predict_outcomes(fighter_1, fighter_2, num_of_fights):
  bout_df = get_bout_params(fighter_1, fighter_2)
  matchesout = []
  results = {"fighter1_dec":0,
  "fighter1_ko":0,
  "fighter1_sub":0,
  "fighter2_dec":0,
  "fighter2_ko":0,
  "fighter2_sub":0}

  for i in range(num_of_fights):
      result = single_fight_simulation(bout_df)
      matchesout.append(result)
      results[result]+=1


  results_table = [["", fighter_1, fighter_2],
                   ["Decision", f"{round((results['fighter1_dec'] / num_of_fights) * 100, 3)}%", f"{round((results['fighter2_dec'] / num_of_fights) * 100, 3)}%"],
                   ["KO", f"{round((results['fighter1_ko'] / num_of_fights) * 100, 3)}%", f"{round((results['fighter2_ko'] / num_of_fights) * 100, 3)}%"],
                   ["Submission", f"{round((results['fighter1_sub'] / num_of_fights) * 100, 3)}%", f"{round((results['fighter2_sub'] / num_of_fights) * 100, 3)}%"]]

  print(tabulate(results_table, headers="firstrow", tablefmt="fancy_grid"))

In [14]:
predict_outcomes("Israel Adesanya", "Sean Strickland", 10000)
print("Actual Result: Sean Strickland (Decision)")

╒════════════╤═══════════════════╤═══════════════════╕
│            │ Israel Adesanya   │ Sean Strickland   │
╞════════════╪═══════════════════╪═══════════════════╡
│ Decision   │ 52.49%            │ 43.24%            │
├────────────┼───────────────────┼───────────────────┤
│ KO         │ 4.12%             │ 0.15%             │
├────────────┼───────────────────┼───────────────────┤
│ Submission │ 0.0%              │ 0.0%              │
╘════════════╧═══════════════════╧═══════════════════╛
Actual Result: Sean Strickland (Decision)


In [15]:
predict_outcomes("Tai Tuivasa", "Alexander Volkov", 10000)
print("Actual Result: Alexander Volkov (Submission)")

╒════════════╤═══════════════╤════════════════════╕
│            │ Tai Tuivasa   │ Alexander Volkov   │
╞════════════╪═══════════════╪════════════════════╡
│ Decision   │ 0.18%         │ 3.21%              │
├────────────┼───────────────┼────────────────────┤
│ KO         │ 39.54%        │ 56.89%             │
├────────────┼───────────────┼────────────────────┤
│ Submission │ 0.0%          │ 0.18%              │
╘════════════╧═══════════════╧════════════════════╛
Actual Result: Alexander Volkov (Submission)


In [16]:
predict_outcomes("Manon Fiorot", "Rose Namajunas", 10000)
print("Actual Result: Manon Fiorot (Decision)")

╒════════════╤════════════════╤══════════════════╕
│            │ Manon Fiorot   │ Rose Namajunas   │
╞════════════╪════════════════╪══════════════════╡
│ Decision   │ 94.15%         │ 0.32%            │
├────────────┼────────────────┼──────────────────┤
│ KO         │ 5.43%          │ 0.03%            │
├────────────┼────────────────┼──────────────────┤
│ Submission │ 0.0%           │ 0.07%            │
╘════════════╧════════════════╧══════════════════╛
Actual Result: Manon Fiorot (Decision)


In [17]:
predict_outcomes("Max Holloway", "Chan Sung Jung", 10000)
print("Actual Result: Max Holloway (KO)")

╒════════════╤════════════════╤══════════════════╕
│            │ Max Holloway   │ Chan Sung Jung   │
╞════════════╪════════════════╪══════════════════╡
│ Decision   │ 6.32%          │ 1.82%            │
├────────────┼────────────────┼──────────────────┤
│ KO         │ 88.0%          │ 3.56%            │
├────────────┼────────────────┼──────────────────┤
│ Submission │ 0.0%           │ 0.3%             │
╘════════════╧════════════════╧══════════════════╛
Actual Result: Max Holloway (KO)
